# <span style="color:darkblue">Using the Geostreams API</span>
As of January 2020 gltg has 32,122,836 datapoints.<br>
<span style="color:red">Please don't fetch all of them at once.</span>
<hr>



In [1]:
import requests
import json
import csv
from csv import DictWriter

### Set general parameters
<span style="color:red">Go to the api_server route to create an account!</span><br>
<span style="color:green">Set server, user credentials, and location for downloads</span>

In [2]:
api_server = r"https://greatlakestogulf.org/geostreams"
user = {'identifier': '******', 'password': '******'}

output_directory = r"downloads"

### Authenticate

In [3]:
r = requests.post(api_server + '/api/authenticate', data=json.dumps(user), headers={'Content-Type': 'application/json'})
print("Authentication status:", r.status_code, "for", api_server)
headers = {"x-auth-token": r.headers["x-auth-token"], "Content-Encoding": "application/json"}

Authentication status: 200 for https://greatlakestogulf.org/geostreams


<hr>

## <span style="color:darkblue">Sensor routes</span>

### Get CSV of all sensors with parameters

In [4]:
sensors = requests.get(api_server + "/api/sensors",headers=headers).json()["sensors"]

print(output_directory + '/gltg_sensors.csv')

with open(output_directory + '/gltg_sensors.csv', 'w') as f:
    fieldnames = [
        'source','name','location', 'longitude', 'latitude', 'max_end_time', 'min_start_time',
        'parameters' , 'huc8', 'huc_name', 'online_status'
    ]

    writer = DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()

    n_sensors = 0
    n_sensors_pos = 0
    for sensor in sensors:
        n_sensors += 1
        parameters_list = []
        for param in sensor['parameters']:
            if param in ['owner','source','unit_code']:
                continue
            if param[-3:] != "-qc":
                parameters_list.append(param + ',\n')
        parameters = "".join(parameters_list)

        huc8 = None
        if 'code' in sensor['properties']['huc']['huc8']:
            huc8 = sensor['properties']['huc']['huc8']['code']
        else:
            huc8 = sensor['properties']['huc']['huc8']

        if len(parameters) == 0:
            n_sensors_pos += 1
            continue

        writer.writerow({
            "source": sensor['properties']['type']['title'],
            "name": sensor['name'],
            "location": sensor['properties'].get('location', ""),
            'longitude': str(sensor['geometry']['coordinates'][0]),
            'latitude': str(sensor['geometry']['coordinates'][1]),
            'max_end_time': sensor.get('max_end_time',''),
            'min_start_time': sensor.get('min_start_time',''),
            'parameters': parameters,
            'huc8': huc8,
            'huc_name': sensor['properties']['huc'].get('huc_name',''),
            'online_status': sensor['properties'].get('online_status',"")
        })
        
print("Sensors skipped " + str(n_sensors_pos) + " of Sensors total " + str(len(sensors)))

downloads/gltg_sensors.csv
Sensors skipped 92 of Sensors total 1296


<hr>

## <span style="color:darkblue">Datapoint Routes</span>

### Download JSON of all datapoints by Sensor ID
<span style="color:green">Set the Sensor ID!</span>

In [5]:
sensor_id = 22

In [6]:
route = api_server + "/api/datapoints?sensor_id=" + str(sensor_id)

r = requests.get(route, headers=headers)

with open(output_directory + '/datapoints_sensor_' + str(sensor_id) + '.json', 'w') as f:
    f.write(json.dumps(r.json(), indent=2))
    
print("Route: " + route)
print("Request Status:", str(r.status_code))
print("Number of datapoints:", len(r.json()))
print("Datapoint JSON saved to " + output_directory + '/datapoints_sensor_' + str(sensor_id) + '.json')

Route: https://greatlakestogulf.org/geostreams/api/datapoints?sensor_id=22
Request Status: 200
Number of datapoints: 107
Datapoint JSON saved to downloads/datapoints_sensor_22.json


### Download CSV of datapoints by sensor ID
<span style="color:green">Set the Sensor ID!</span>

In [7]:
sensor_id = 22

In [8]:
route = api_server + "/api/datapoints?sensor_id=" + str(sensor_id) + "&format=csv"

r = requests.get(route, headers=headers)

with open(output_directory + '/datapoints_sensor_' + str(sensor_id) + '.csv', 'w') as f:
    f.write(r.text)
    
print("Route: " + route)
print("Request Status:", str(r.status_code))
print("Datapoint JSON saved to " + output_directory + '/datapoints_sensor_' + str(sensor_id) + '.csv')

Route: https://greatlakestogulf.org/geostreams/api/datapoints?sensor_id=22&format=csv
Request Status: 200
Datapoint JSON saved to downloads/datapoints_sensor_22.csv
